# Predictive Analytics

## Preparation

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go

In [17]:
Rides = pd.read_csv("chicago_2018.csv", sep=",")

Rides["start_time"] = pd.to_datetime(Rides["start_time"])
Rides["end_time"] = pd.to_datetime(Rides["end_time"])

Rides.sort_values("start_time", inplace = True)
Rides.head(8)

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
3215937,2018-01-01 00:12:00,2018-01-01 00:17:23,69,159,Damen Ave & Pierce Ave,Claremont Ave & Hirsch St,3304,Subscriber
3215938,2018-01-01 00:41:35,2018-01-01 00:47:52,253,325,Winthrop Ave & Lawrence Ave,Clark St & Winnemac Ave (Temp),5367,Subscriber
3215939,2018-01-01 00:44:46,2018-01-01 01:33:10,98,509,LaSalle St & Washington St,Troy St & North Ave,4599,Subscriber
3215940,2018-01-01 00:53:10,2018-01-01 01:05:37,125,364,Rush St & Hubbard St,Larrabee St & Oak St,2302,Subscriber
3215941,2018-01-01 00:53:37,2018-01-01 00:56:40,129,205,Blue Island Ave & 18th St,Paulina St & 18th St,3696,Subscriber
3215942,2018-01-01 00:56:15,2018-01-01 01:00:41,304,299,Broadway & Waveland Ave,Halsted St & Roscoe St,6298,Subscriber
3215943,2018-01-01 00:57:26,2018-01-01 01:02:40,164,174,Franklin St & Lake St,Canal St & Madison St,1169,Subscriber
3215944,2018-01-01 01:00:29,2018-01-01 01:13:43,182,142,Wells St & Elm St,McClurg Ct & Erie St,6351,Subscriber


The following cell is just for double checking the aggregated, hourly rental counts.

In [18]:
date1 = datetime.datetime(year=2018, month=1, day=1, hour = 5)
date2 = datetime.datetime(year=2018, month=1, day=1, hour = 6)

Rides[(Rides["start_time"] >= date1) & (Rides["start_time"] <= date2)]

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
3215973,2018-01-01 05:32:22,2018-01-01 05:40:04,206,339,Halsted St & Archer Ave,Emerald Ave & 31st St,540,Subscriber
3215974,2018-01-01 05:46:51,2018-01-01 05:58:48,72,36,Wabash Ave & 16th St,Franklin St & Jackson Blvd,3775,Subscriber
3215975,2018-01-01 05:48:40,2018-01-01 05:53:01,303,296,Broadway & Cornelia Ave,Broadway & Belmont Ave,4784,Subscriber


Computing the hourly demand for 2018. The result is stored in a DataFrame of 24 by 365 = 8760 rows, neglecting time change

In [19]:
date_index = date1 + pd.to_timedelta(np.arange(8760), 'H')

Features = pd.DataFrame(index = date_index)

Features["demand"] = 0

Features = Rides.set_index("start_time")
Features = Features.resample('H').count()

Features.drop(columns = "start_station_id", inplace = True)
Features.drop(columns = "end_station_id", inplace = True)
Features.drop(columns = "start_station_name", inplace = True)
Features.drop(columns = "end_station_name", inplace = True)
Features.drop(columns = "bike_id", inplace = True)
Features.drop(columns = "user_type", inplace = True)

Features.rename(columns = {"end_time": "Rides"}, inplace = True)

The resulting df can be double checked with one of the cells above or with the sorted Rides df. It seems reasonable though:

In [20]:
Features.describe()

,Rides
count,8760.000000
mean,411.310731
std,479.667282
min,0.000000
25%,55.000000
50%,225.000000
75%,611.250000
max,2829.000000


In [21]:
Features.head()

,Rides
start_time,
2018-01-01 00:00:00,7
2018-01-01 01:00:00,15
2018-01-01 02:00:00,10
2018-01-01 03:00:00,2
2018-01-01 04:00:00,2


In [22]:
x = Features.iloc[0]["Rides"]
x

7

In [23]:
Features["Rides_last_hour"] = 0
value = Features.iloc[0]["Rides"]
for i in Features.index:     
        Features.loc[i,"Rides_last_hour"] = value
        value = Features.loc[i]["Rides"]

There seems to be some erroneous data in the weather data set as there are rows which exhibit the same date, leading pandas to crash. (e.g. for index 1662, if duplicates were not removed.)

In [24]:
Weather = pd.read_csv("weather_hourly_chicago.csv", sep=",")
Weather.head(5)

,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,-1.7,-1.7,0.0
1,2015-01-02 02:00:00,-2.2,-2.2,0.0
2,2015-01-02 03:00:00,-2.8,-2.8,0.0
3,2015-01-02 04:00:00,-3.3,-3.3,0.0
4,2015-01-02 05:00:00,-4.4,-4.4,0.0


In [25]:
Weather.tail(5)

,date_time,max_temp,min_temp,precip
43843,2020-01-01 20:00:00,4.4,4.4,0.0
43844,2020-01-01 21:00:00,5.0,5.0,0.0
43845,2020-01-01 22:00:00,5.0,5.0,0.0
43846,2020-01-01 23:00:00,4.4,4.4,0.0
43847,2020-01-02 00:00:00,4.4,4.4,0.0


In [26]:
len(Weather)

43848

In [27]:
Weather.isna().sum()

date_time    60
max_temp     60
min_temp     60
precip       58
dtype: int64

In [28]:
Weather["avg_tmp"] = (Weather["max_temp"]+Weather["min_temp"])/2
Weather["is_raining"] = Weather["precip"] == 1
Weather.drop(columns = ["max_temp", "min_temp", "precip"], inplace=True)

Weather["date_time"] = pd.to_datetime(Weather["date_time"])

Weather.set_index("date_time", inplace = True)
Weather.head(5)

,avg_tmp,is_raining
date_time,,
2015-01-02 01:00:00,-1.7,False
2015-01-02 02:00:00,-2.2,False
2015-01-02 03:00:00,-2.8,False
2015-01-02 04:00:00,-3.3,False
2015-01-02 05:00:00,-4.4,False


In [ ]:
Features = Features.join(Weather, on="start_time")

Lots of missing values for weather, imputation methods:

* numerical values: linear interpolation (time series data)
* categorical (is_raining): backwards-fill

In [ ]:
Features.interpolate(inplace=True)
Features.fillna(method="bfill",inplace=True)

In [ ]:
Features[Features["is_raining"].isnull()]

In [ ]:
len(Features[Features["is_raining"] == True])

In [ ]:
Features.describe()

In [ ]:
Features.reset_index(inplace=True)

In [ ]:
Features["is_workday"] = Features["start_time"].apply(lambda x: x.weekday() < 5)
Features["hour"] = Features["start_time"].apply(lambda x: x.hour)
Features["month"] = Features["start_time"].apply(lambda x: x.month)

In [ ]:
def getSeason(month):
    
    Winter = [12, 1, 2]
    Spring = [3, 4, 5]
    Summer = [6, 7, 8]
    Fall = [9, 10, 11]
    
    if month in Winter:
        return 1
    elif month in Spring:
        return 2
    elif month in Summer:
        return 3
    elif month in Fall:
        return 4
    
Features["season"] = Features["month"].apply(lambda month: getSeason(month))

In [ ]:
seasons = pd.get_dummies(Features["season"],prefix="season_")
seasons.drop(columns="season__4", inplace=True)

In [ ]:
Features[list(seasons.columns)] = seasons

In [ ]:
hours = pd.get_dummies(Features["hour"],prefix="hour_")
hours.drop(columns="hour__23", inplace=True)

In [ ]:
Features[list(hours.columns)] = hours

In [ ]:
Features.drop(columns=["season","month","hour"], inplace=True)

In [ ]:
Features

Re-Scaling the data

In [ ]:
Features_rescaled = pd.DataFrame()

Features_rescaled["Rides"] = (Features["Rides"] - Features["Rides"].min()) / (Features["Rides"].max() - Features["Rides"].min())
Features_rescaled["Rides_last_hour"] = (Features["Rides_last_hour"] - Features["Rides_last_hour"].min()) / (Features["Rides_last_hour"].max() - Features["Rides_last_hour"].min())
Features_rescaled["Max_temp"] = (Features["Max_temp"] - Features["Max_temp"].min()) / (Features["Max_temp"].max() - Features["Max_temp"].min())
Features_rescaled["Min_temp"] = (Features["Min_temp"] - Features["Min_temp"].min()) / (Features["Min_temp"].max() - Features["Min_temp"].min())
Features_rescaled["Precipitation"] = (Features["Precipitation"] - Features["Precipitation"].min()) / (Features["Precipitation"].max() - Features["Precipitation"].min())
Features_rescaled["Day_of_Week"] = (Features["Day_of_Week"] - Features["Day_of_Week"].min()) / (Features["Day_of_Week"].max() - Features["Day_of_Week"].min())
Features_rescaled["Hour"] = (Features["Hour"] - Features["Hour"].min()) / (Features["Hour"].max() - Features["Hour"].min())
Features_rescaled["Month"] = (Features["Month"] - Features["Month"].min()) / (Features["Month"].max() - Features["Month"].min())
Features_rescaled["Season"] = (Features["Season"] - Features["Season"].min()) / (Features["Season"].max() - Features["Season"].min())

Features_rescaled.head()

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=Features["start_time"], y=Features_rescaled["Rides"],
                    mode='lines',
                    name='Demand'))
fig.add_trace(go.Scatter(x=Features["start_time"], y=Features_rescaled["Max_temp"],
                    mode='lines+markers',
                    name='Max. Temperature'))

fig.show()

In [ ]:
sns.jointplot(x = "avg_tmp", y = "Rides", data = Features, kind = "hex", height=10, palette = "magma")

In [ ]:
sns.pairplot(Features, palette="magma", height=3, hue="is_raining")
plt.show()

In [ ]:
Features.cov()

In [ ]:
Features_corr = Features.corr()
Features_corr

In [ ]:
sns.heatmap(Features_corr, 
        xticklabels=Features_corr.columns,
        yticklabels=Features_corr.columns)

[Source](https://stackoverflow.com/questions/39409866/correlation-heatmap) for the following code:

In [ ]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

Features_corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
Features_rescaled.var()

In [ ]:
Features_sample = Features[3100:3220]
Features_sample = Features_sample[Features["avg_tmp"] != 0]
#Features_sample = Features.sample(n=125)
#Features_sample = Features_sample[Features["Max_temp"] != 0]

sns.scatterplot(x = Features_sample["avg_tmp"], y = Features_sample["Rides"], hue = Features_sample["is_raining"])

In [ ]:
fig = px.density_heatmap(Features, x='avg_tmp', y='Rides', width=600, height=600,
                      title='Correlation between Temperature and Demand', color_continuous_scale=[[0.0, 'white'], [1.0, 'red']],
                        nbinsx=25, nbinsy=25)
fig.show()

In [ ]:
fig = px.parallel_coordinates(data_frame = Features, dimensions =[""])

fig.show()

In [ ]:
fig = px.scatter_3d(data_frame = Features_sample, z='Max_temp', x='Rides', y='Hour', color='Precipitation', opacity=0.4, size_max=5)

fig.show()

In [ ]:
fig = px.scatter_3d(data_frame = Features_sample, z='Max_temp', x='Rides', y='Rides_last_hour', color='Precipitation', opacity=0.4, size_max=5)

fig.show()

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split the data with 70-30% split as above

X = Features[Features.columns[(Features.columns != "Rides") & (Features.columns != "start_time")]]
y = Features[["Rides"]]
X,y
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test

fangt hier an...

### Multi-dimensional linear regression

Let's now try to train a mulit-dimensional linear model. But first of all, we need to import a few missing libraries.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In order to be able to use sklearn, we need to eliminate booleans in our data first.

In [ ]:
def workday_check (dt):
    
    if dt == True:
        return 1
    else:
        return 0

Features["is_workday"] = Features["is_workday"].apply(lambda dt: workday_check(dt))

def raining_check (dt):
    
    if dt == True:
        return 1
    else:
        return 0

Features["is_raining"] = Features["is_raining"].apply(lambda dt: raining_check(dt))

In [ ]:
# split the data with 70-30% split as above

X = Features[Features.columns[(Features.columns != "Rides") & (Features.columns != "start_time")]]
y = Features[["Rides"]]
X,y
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X

Now the same for average temperature and precipitation

In [ ]:
x_train2 = x_train[["is_raining","avg_tmp"]]  #Fist select only two features for the sake of simlicity
lin_mod_day = LinearRegression()
lin_mod_day.fit(x_train2,y_train)
print(lin_mod_day.coef_, lin_mod_day.intercept_)

print()

In [ ]:
X_raining = x_train2[x_train2["is_raining"]==1]
X_nraining = x_train2[x_train2["is_raining"]==0]

y_pred_raining = lin_mod_day.predict(X_raining)
y_pred_nraining = lin_mod_day.predict(X_nraining)

plt.figure(figsize = (16,9))

plt.scatter(X_nraining["avg_tmp"], 
            y_train[x_train2["is_raining"]==False]["Rides"], 
            marker="x", label="Not raining")

plt.scatter(X_raining["avg_tmp"], 
            y_train[x_train2["is_raining"]==True]["Rides"], 
            marker="+", label="Raining")


plt.plot(X_raining["avg_tmp"], 
         y_pred_raining,color="orange", 
         label="Raining Prediction")

plt.plot(X_nraining["avg_tmp"], 
         y_pred_nraining, 
         label="Not raining Prediction")

plt.xlabel("Average Temperature")
plt.ylabel("Rides")

plt.legend()
plt.show()

Ignore everything above, now the correct stuff

In [ ]:
lin_mod = LinearRegression()
lin_mod.fit(x_train,y_train)
print(lin_mod_day.coef_, lin_mod_day.intercept_)

v = x_test.values
y_true = y_test
y_pred = lin_mod.predict(v)

In [ ]:
print("Mean Squared Error:",mean_squared_error(y_pred, y_true),"Rides^2")
print("Root Mean Squared Error:",mean_squared_error(y_pred, y_true)**0.5,"Rides")
print("Mean Absolute Error:",mean_absolute_error(y_pred, y_true),"Rides")
print("Coefficient of determination:",r2_score(y_pred, y_true))

Let's try to get a better R^2 score by reducing the input features

In [ ]:
# split the data with 70-30% split as above

X = Features[Features.columns[(Features.columns != "Rides") & (Features.columns != "start_time") & (Features.columns != "Rides_last_hour")]]
y = Features[["Rides"]]
X,y
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X

In [ ]:
lin_mod = LinearRegression()
lin_mod.fit(x_train,y_train)
print(lin_mod_day.coef_, lin_mod_day.intercept_)

v = x_test.values
y_true = y_test
y_pred = lin_mod.predict(v)

In [ ]:
print("Mean Squared Error:",mean_squared_error(y_pred, y_true),"Rides^2")
print("Root Mean Squared Error:",mean_squared_error(y_pred, y_true)**0.5,"Rides")
print("Mean Absolute Error:",mean_absolute_error(y_pred, y_true),"Rides")
print("Coefficient of determination:",r2_score(y_pred, y_true))

In this particular feature configuartion, leaving out the rides_last_hour feature significantly decreases the R^2 score by around 38 percentage points. Thus let's try some more configurations.

In [ ]:
# split the data with 70-30% split as above

X = Features[Features.columns[(Features.columns != "Rides") & (Features.columns != "start_time") & (Features.columns != "is_workday")]]
y = Features[["Rides"]]
X,y
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X

In [ ]:
lin_mod = LinearRegression()
lin_mod.fit(x_train,y_train)
print(lin_mod_day.coef_, lin_mod_day.intercept_)

v = x_test.values
y_true = y_test
y_pred = lin_mod.predict(v)

In [ ]:
print("Mean Squared Error:",mean_squared_error(y_pred, y_true),"Rides^2")
print("Root Mean Squared Error:",mean_squared_error(y_pred, y_true)**0.5,"Rides")
print("Mean Absolute Error:",mean_absolute_error(y_pred, y_true),"Rides")
print("Coefficient of determination:",r2_score(y_pred, y_true))

As seen above, leaving out the is_workday feature does not affect our R^2 score that much. It only decreases it by around 0.001 percentage points.

In [ ]:
# split the data with 70-30% split as above

X = Features[Features.columns[(Features.columns != "Rides") & (Features.columns != "start_time") ]]
y = Features[["Rides"]]
X,y
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)
X



lin_mod = LinearRegression()
lin_mod.fit(x_train,y_train)
print(lin_mod_day.coef_, lin_mod_day.intercept_)

v = x_test.values
y_true = y_test
y_pred = lin_mod.predict(v)



print("Mean Squared Error:",mean_squared_error(y_pred, y_true),"Rides^2")
print("Root Mean Squared Error:",mean_squared_error(y_pred, y_true)**0.5,"Rides")
print("Mean Absolute Error:",mean_absolute_error(y_pred, y_true),"Rides")
print("Coefficient of determination:",r2_score(y_pred, y_true))